In [1]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import date, timedelta, datetime, timezone
import time
import pytz
import json
from os import path
from urllib.parse import unquote
import base64

In [2]:
with open("./apikeys.key") as APIfp:
    obj = json.load(APIfp)
    # financialKey = obj['financialAPI']
    SchwabConsumer = obj["Schwab_consumer"]
    SchwabSecret = obj["Schwab_secret"]
    SchwabRefresh = obj['Schwab_refresh']

In [3]:
def retrieveTDAuthToken():
    """Get auth token using refresh token"""
    client_encoded = base64.b64encode(f"{SchwabConsumer}:{SchwabSecret}".encode("utf-8")).decode("utf-8")
    r = requests.post("https://api.schwabapi.com/v1/oauth/token",
    headers={
        "Authorization": f"Basic {client_encoded}",
        "Content-Type": 'application/x-www-form-urlencoded',},
    data={
        'grant_type': 'refresh_token',
        'refresh_token': SchwabRefresh,
        'redirect_uri':'https://127.0.0.1'

    })
    if r.status_code != 200:
        print("Error getting TD auth code")
        print(r.content)
    else:
        print("success retrieving token")
    global TDAuthToken, TDAuthExpire
    TDAuthToken = json.loads(r.text)["access_token"]
    TDAuthExpire = datetime.now() + timedelta(minutes=29)


In [5]:
retrieveTDAuthToken()
print(TDAuthToken)

success retrieving token
I0.b2F1dGgyLmNkYy5zY2h3YWIuY29t.lcrJ3lyxK8GpWU04SgeuCRkY7pg4tDrTFXoyeJ6miLk@


In [22]:
def getSymbolsTD(arr: list, directory="data", freqType="minute", periodType="day", freq=1, period=5, endDate=None, daysBack=0, save=True,
                 disableDelay=False, datedFileName=False):
    if endDate is None:
        # Calculate end time as now, or previous close if it is past close
        prevclose = datetime.now(tz=pytz.timezone("America/New_York"))
        if prevclose.hour < 4:
            prevclose -= timedelta(days=1)
        endDateStamp = 1000 * int(min(time.time(), prevclose.timestamp()))
        endDate = datetime.fromtimestamp(endDateStamp / 1e3)
    else:
        endDateStamp = 1000 * int(endDate.timestamp())
    if daysBack > 0 and endDate.weekday() < daysBack:
        daysBack += 2
    startDateStamp = endDateStamp - (daysBack * 86400000)

    res = {}
    count = 0
    errors = 0
    for sym in arr:
        count += 1
        if not disableDelay:
            print(f"Downloading  {sym}", end=" ")
        fileName = f"{directory}/{sym}{endDate.strftime('%m-%d-%Y') if datedFileName else ''}.csv"
        if save and path.exists(fileName):
            print("File exists")
            continue
        if not disableDelay:
            print("")
        nextRun = time.time() + 0.51
        r = requests.get("https://api.schwabapi.com/marketdata/v1/pricehistory",
                         headers={"Authorization": f"Bearer {TDAuthToken}"},
                         params={
                             "symbol":sym,
                             "periodType": periodType,
                             "frequencyType": freqType,
                             "frequency": freq,
                             # In place of end start
                             "period": period
                             #  "needExtendedHoursData": "true",
                             #  "endDate": endDateStamp,
                             #  "startDate": startDateStamp,
                         })
        if r.status_code == 200:
            try:
                obj = json.loads(r.text)
                df = pd.DataFrame(obj["candles"])
                df["datetime"] = pd.to_datetime(df['datetime'], unit='ms').dt.tz_localize(
                    'UTC').dt.tz_convert('America/New_York')
                if save:
                    df.to_csv(fileName, index=False)
                else:
                    res[sym] = df
            except Exception as e:
                print("Download Error ", sym)
                print(e)
                errors += 1
        else:
            print(f"Download Error {sym} \n {r.text}")
            errors += 1
        sleepDur = nextRun - time.time()
        if sleepDur > 0 and not disableDelay:
            time.sleep(sleepDur)
    print(f"Total #errors {errors}")
    if not save:
        return res

In [25]:
getSymbolsTD(['AAPL'], directory="./", freqType="minute", periodType="day", period=10)

Total #errors 0
